In [1]:
import pandas as pd
import numpy as np

## Can. Parl. Dataset

In [2]:
data_file = '../DG_data/CanParl/ml_CanParl.csv'
data_df = pd.read_csv(data_file)
data_df = data_df.drop(axis=1, columns=['Unnamed: 0'])
data_df['ts'].value_counts().sort_index()

0.0            3734
31536000.0     5501
63072000.0     3682
94694400.0     5130
126230400.0    4409
157766400.0    3109
189302400.0    8334
220924800.0    6631
252460800.0    7729
283996800.0    4297
315532800.0    4844
347155200.0    6965
378691200.0    7015
410227200.0    3098
Name: ts, dtype: int64

In [3]:
time_granularity = 2 # original unique steps / coarse unique steps

In [4]:
num_unique_step = len(data_df['ts'].value_counts())
num_unique_step = num_unique_step // time_granularity
num_unique_step

7

### Quantile Split

In [5]:
# if aggregate with quantile, the test set (15%) might be null
quantile_prop = 1 / num_unique_step
quantile_prop_list = [quantile_prop * i for i in range(num_unique_step + 1)]
quantile_list = [data_df['ts'].quantile(q) for q in quantile_prop_list]
quantile_list

[0.0,
 63072000.0,
 126230400.0,
 189302400.0,
 252460800.0,
 315532800.0,
 347155200.0,
 410227200.0]

In [6]:
for i in range(len(quantile_list) - 1):
    # if i == len(quantile_list) - 2:
    #     data_df.loc[(data_df['ts'] >= quantile_list[i])&(data_df['ts'] <= quantile_list[i + 1]), 'ts'] = quantile_list[i]
    # else:
    data_df.loc[(data_df['ts'] >= quantile_list[i])&(data_df['ts'] < quantile_list[i + 1]), 'ts'] = quantile_list[i]
data_df['ts'].value_counts().sort_index()

0.0             9235
63072000.0      8812
126230400.0     7518
189302400.0    14965
252460800.0    12026
315532800.0     4844
347155200.0    13980
410227200.0     3098
Name: ts, dtype: int64

In [7]:
data_df.to_csv('../DG_data_coarse/ml_CanParl.csv')